In [32]:
import praw
import csv
import re
import json   
import requests
import time
import mysql.connector

reddit = praw.Reddit()

class StockPost(object):
    def __init__(self, postID, postURL, ups, downs, numComments, stock):
        self.postID = postID
        self.url = postURL
        self.stock = stock
        self.ups = ups
        self.downs = downs
        self.numComments = numComments
    
    def jsonEnc(self):
      return {'stock': self.stock, 'postID': self.postID, 'postURL': self.url, 'ups': self.ups, 'downs': self.downs, 'numComments': self.numComments}

def jsonDefEncoder(obj):
    if hasattr(obj, 'jsonEnc'):
        return obj.jsonEnc()
    else: #some default behavior
        return obj.__dict__
    
    ## returns cursor object ##
def connect_to_db(db_name):
    cnx = mysql.connector.connect(
    user='root',
    password='chalkHorseMountain',
    host='localhost',
    database=db_name
    )
    return cnx.cursor()
    
    ## returns boolean ##
def table_exists(tbl_name):
    mycursor.execute("""
        SELECT COUNT(*)
        FROM information_schema.tables
        WHERE table_schema = DATABASE()
        AND table_name = \"""" + tbl_name + """\";
    """)
    
    if mycursor.fetchone()[0] == 1:
        return True
    return False

class SubredditScraper:

    def __init__(self, sub, sort='new', lim=900):
        self.sub = sub
        self.sort = sort
        self.lim = lim

        #print(
            #f'SubredditScraper instance created with values '
            #f'sub = {sub}, sort = {sort}, lim = {lim}')

    def set_sort(self):
        if self.sort == 'new':
            return self.sort, reddit.subreddit(self.sub).new(limit=self.lim)
        elif self.sort == 'top':
            return self.sort, reddit.subreddit(self.sub).top(limit=self.lim)
        elif self.sort == 'hot':
            return self.sort, reddit.subreddit(self.sub).hot(limit=self.lim)
        else:
            self.sort = 'hot'
            print('Sort method was not recognized, defaulting to hot.')
            return self.sort, reddit.subreddit(self.sub).hot(limit=self.lim)

    def get_posts(self):

        stockTickers = {}
        with open('tickers_stocks.csv', mode='r') as infile:
            reader = csv.reader(infile)
            for row in reader:
                stockTickers[row[0].split(',')[0]] = {}
        """Get unique posts from a specified subreddit."""

        # Attempt to specify a sorting method.
        sort, subreddit = self.set_sort()

        print(f'Collecting information from r/{self.sub}.')
        
        ## Search posts for tickers ##
        relevantPosts = []
        i = 0
        for post in subreddit:
            i = i + 1
            print(f'{i}/{self.lim}')
            if post.link_flair_text != 'Meme':
                for stock in stockTickers.keys():
                    if(re.search(r'\s+\$?' + stock + r'\$?\s+', post.selftext) or re.search(r'\s+\$?' + stock + r'\$?\s+',  post.title)):
                        stockTickers[stock][post.id] = StockPost(post.id, post.permalink, post.ups, post.downs, post.num_comments, stock)
        
        ## Upload data to db ##
        mycursor = connect_to_db("redditScraper")
        assert table_exists("reddit")
        
        for stock in stockTickers.keys():
            if (len(stockTickers[stock]) > 0):
                for post in stockTickers[stock]:
                    
                    #relevantPosts.append(stockTickers[stock][post]) 
        #json_object = json.dumps(relevantPosts, default=jsonDefEncoder, indent = 4)   
        #print(json_object)  


if __name__ == '__main__':
    bef = time.perf_counter()
    SubredditScraper('wallstreetbets', lim=20, sort='hot').get_posts()
    aft = time.perf_counter()
    print(aft-bef)
    

1/20
2/20
3/20
4/20
5/20
6/20
7/20
8/20
9/20
10/20
11/20
12/20
13/20
14/20
15/20
16/20
17/20
18/20
19/20
20/20
[
    {
        "stock": "CLOV",
        "postID": "n77rdn",
        "postURL": "/r/wallstreetbets/comments/n77rdn/god_tier_dd_buy_now_get_rich_monday/",
        "ups": 7891,
        "downs": 0,
        "numComments": 888
    },
    {
        "stock": "CLOV",
        "postID": "n7um6w",
        "postURL": "/r/wallstreetbets/comments/n7um6w/clov_will_be_a_multibagger/",
        "ups": 98,
        "downs": 0,
        "numComments": 112
    },
    {
        "stock": "MVIS",
        "postID": "n77rdn",
        "postURL": "/r/wallstreetbets/comments/n77rdn/god_tier_dd_buy_now_get_rich_monday/",
        "ups": 7891,
        "downs": 0,
        "numComments": 888
    },
    {
        "stock": "PT",
        "postID": "n7um6w",
        "postURL": "/r/wallstreetbets/comments/n7um6w/clov_will_be_a_multibagger/",
        "ups": 98,
        "downs": 0,
        "numComments": 112
    },
   

In [ ]:
## Creating reddit_db
mycursor.execute("""
    CREATE TABLE reddit (
    post_id INT,
    symbol VARCHAR(30),
    num_comments INT,
    num_votes INT,
    date_posted DATE
    );
    """)

In [ ]:
## make "post_id" and "symbol" composite primary keys
mycursor.execute("""
    ALTER TABLE reddit 
    ADD PRIMARY KEY (post_id, symbol)
    ;""")